In [3]:
# Function calling
!pip install requests

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-win_amd64.whl.metadata (36 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp313-cp313-win_amd64.whl (105 kB)

   ---------------------------------------- 0/3 [urllib3]
   ---------------------------------------- 0/3 [urllib3]
   ---------------------------------------- 0/3 [urllib3]
   ---------------------------------------- 0/3 [urllib3]
   ------------- -------------------------- 1/3 [charset_normalizer]
   ------------- -------------------------- 1/3 [charset_normalizer]
   -------------------------- ------------- 2/3 [requests]
   -------------------------- ------------- 2/3 [requests]
   ---------------------------------------- 3/3 [requests]



In [1]:
import requests
import datetime
import inspect
import json
from typing import (
    TypedDict, 
    List, Dict, Literal, 
    Callable, Optional, Any, 
    get_type_hints
)
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=api_key
)


In [ ]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], 
                    content: str,
                    tool_calls: dict=dict(),        #dict = dict() sets a default empty dictionary orrrrrrrrrr u can use {}
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

In [ ]:
# Storing user question and assistant generated message in the memory

def chat_with_tools(user_question:str=None, memory:Memory=None, model:str="gpt-4o-mini", temperature=0.0, tools=None)-> str:

    messages = [{"role": "user", "content": user_question}]
    if memory:
        if user_question:
            memory.add_message(role="user", content=user_question)
        messages = memory.get_messages()        

    
    response = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
        tools=tools,
    )
    
    ai_message = str(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls
    
    if memory:
        memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)
    
    return ai_message

In [7]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [12]:
# it's used for tool/function calling in OpenAI's GPT API.
tools = [{
    "type": "function",
    "function": {
        "name": "power",
        "description": "Exponentatiation: base to the power of exponent",
        "parameters": {
            "type": "object",
            "properties": {
                "base": {"type": "number"},
                "exponent": {"type": "number"}
            },
            "required": ["base", "exponent"],
            "additionalProperties": False
        },
        "strict": True
    }
}]



# Backend json File................................
        # {
        # "role": "assistant",
        # "tool_calls": [
        #     {
        #     "id": "call_abc123",
        #     "type": "function",
        #     "function": {
        #         "name": "power",
        #         "arguments": "{\"base\": 2, \"exponent\": -5}"
        #     }
        #     }
        # ]
        # }

In [ ]:
# Instantiate memory and start with the system prompt
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

# Call the LLM with a question that needs a tool
ai_message = chat_with_tools(
    user_question="2 to the power of -5?",
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

# args = memory.last_message()['tool_calls'][0].function.arguments
# print(args)
# #tool_call_id = json.loads(memory.last_message()['tool_calls'][0].id)
# tool_call_id = memory.last_message()['tool_calls'][0].id
# print(tool_call_id)

# Get the arguments from the tool_calls object and call the actual defined function
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
result = power(args["base"], args["exponent"])

# Extract the tool_call_id and feed the LLM with the result from the function 

tool_call_id = memory.last_message()['tool_calls'][0].id
memory.add_message(
    role="tool",
    content=str(result), 
    tool_call_id=tool_call_id
)

ai_message = chat_with_tools(

    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

{"base":2,"exponent":-5}
call_m9MtVoJ6mgZpRMqyBYoMzOMK


In [ ]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_BdcLmiuO7OWiY0WnyJ07n9jr', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_BdcLmiuO7OWiY0WnyJ07n9jr'},
 {'role': 'assistant',
  'content': '2 to the power of -5 is equal to 0.03125.',
  'tool_calls': None}]

In [ ]:
class Tool:
    def __init__(self, func:Callable):
        self.func = func
        self.name = func.__name__
        self.description = func.__doc__
        self.argument_types_map = get_type_hints(func)
        self.signature = inspect.signature(func)
        self.arguments = [
            {
                "name": key, 
                "type": self._infer_json_schema_type(value),
                "required": param.default == inspect.Parameter.empty
            } 
            for key, value in self.argument_types_map.items()
            if (param := self.signature.parameters.get(key))
        ]

    def dict(self):
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parallel_tool_calls": False,
                "parameters": {
                    "type": "object",
                    "properties": {
                        argument["name"]: {
                            "type": argument["type"],
                        }
                        for argument in self.arguments
                    },
                    "required": [
                        argument["name"] 
                        for argument in self.arguments 
                        if argument["required"]
                    ],
                    "additionalProperties": False,
                },
                "strict": True
            }
        }

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    
    def _infer_json_schema_type(self, arg_type: Any) -> str:
        if arg_type == bool:
            return "boolean"
        elif arg_type == int:
            return "integer"
        elif arg_type == float:
            return "number"
        elif arg_type == str:
            return "string"
        elif arg_type == list:
            return "array"
        elif arg_type == dict:
            return "object"
        elif arg_type is None:
            return "null"
        elif arg_type == datetime.date or arg_type == datetime.datetime:
            return "string"  # JSON Schema treats dates as strings
        else:
            return "string"  # Default to string if type is unknown

In [ ]:
power_tool = Tool(power)

In [ ]:
power_tool.dict()

{'type': 'function',
 'function': {'name': 'power',
  'description': 'Exponentatiation: base to the power of exponent',
  'parallel_tool_calls': False,
  'parameters': {'type': 'object',
   'properties': {'base': {'type': 'number'}, 'exponent': {'type': 'number'}},
   'required': ['base', 'exponent'],
   'additionalProperties': False},
  'strict': True}}

In [ ]:
power_tool(2,3)

8

In [ ]:
#Update the Agent class

# You will modify the logic responsible for:

    # Processing user input – The agent should record and manage conversation history.
    # Generating a response – The agent will use a language model to create a reply based on previous messages.
    # Identifying when tools are needed – If a tool is required to complete the request, the agent should detect this and trigger the appropriate function.
    # Handling tool execution and responses – The agent should execute the tool, capture its output, and integrate the result into the conversation.


In [ ]:
class Agent:
    """A tool-calling AI Agent"""

    def __init__(
        self,
        name:str = "Agent", 
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
        tools:List[Tool] = [],
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, " 
                    f"and you need to {self.instructions}",
        )

        self.client = client

        self.tools = tools
        self.tool_map = {t.name:t for t in tools}
        self.openai_tools = [t.dict() for t in self.tools] if self.tools else None

    def invoke(self, user_message: str) -> str:
        self.memory.add_message(
            role="user",
            content=user_message,
        )

        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )

        tool_calls = ai_message.tool_calls
        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=tool_calls,
        )

        if tool_calls:
            self._call_tools(tool_calls)
            
        return self.memory.last_message()

    def _call_tools(self, tool_calls:List[ChatCompletionMessageToolCall]):
        for t in tool_calls:
            tool_call_id = t.id
            function_name = t.function.name
            args = json.loads(t.function.arguments)
            callable_tool = self.tool_map[function_name]
            result = callable_tool(**args)
            self.memory.add_message(
                role="tool", 
                content=str(result), 
                tool_call_id=tool_call_id
            )

        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )

        tool_calls = ai_message.tool_calls

        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=tool_calls,
        )

        if tool_calls:
            self._call_tools(tool_calls)


    def _get_completion(self, messages:List[Dict])-> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages,
            tools=self.openai_tools,
        )
        
        return response.choices[0].message


In [ ]:
# Build some agents and have fun

In [ ]:
agent = Agent(
    tools=[Tool(power)]
)

In [ ]:
agent.invoke("What is 10 + 5?")

{'role': 'assistant', 'content': '10 + 5 equals 15.', 'tool_calls': None}

In [ ]:
agent.memory.get_messages()

[{'role': 'system',
  'content': "You're an AI Agent, your role is Personal Assistant, and you need to Help users with any question",
  'tool_calls': {}},
 {'role': 'user', 'content': 'What is 10 + 5?', 'tool_calls': {}},
 {'role': 'assistant', 'content': '10 + 5 equals 15.', 'tool_calls': None}]

In [ ]:
agent.memory.reset()

In [ ]:
agent.memory.get_messages()

[]

In [ ]:
agent.invoke("What is 3 to the power of (2 to the power of 2)?")

{'role': 'assistant',
 'content': '3 to the power of (2 to the power of 2) is 81.',
 'tool_calls': None}

In [ ]:
agent.memory.get_messages()

[{'role': 'user',
  'content': 'What is 3 to the power of (2 to the power of 2)?',
  'tool_calls': {}},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_syuRuVri1qJOo7E94WILwQvz', function=Function(arguments='{"base": 2, "exponent": 2}', name='power'), type='function'),
   ChatCompletionMessageToolCall(id='call_iCKqw9xMy7U1WLOmEajmpxWP', function=Function(arguments='{"base": 3, "exponent": 0}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '4',
  'tool_call_id': 'call_syuRuVri1qJOo7E94WILwQvz'},
 {'role': 'tool',
  'content': '1',
  'tool_call_id': 'call_iCKqw9xMy7U1WLOmEajmpxWP'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_37HTEVJN2SIbQ5y5yh4OLITp', function=Function(arguments='{"base":3,"exponent":4}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '81',
  'tool_call_id': 'call_37HTEVJN2SIbQ5y5yh4OLITp'},
 {'role': 'assistant',
  'content': '3

# Additional 

In [ ]:
"""
| Term         | Direction | Appears in        |
| ------------ | --------- | ----------------- |
| `tools`      | You ➜ LLM | **Request** only  |  What tools/functions you give to the LLM            input
| `tool_calls` | LLM ➜ You | **Response** only |  What tools the LLM decided to call                  output

"""

In [ ]:
"""
response = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)
"""
#############################################
# OpenAi Response for above code looks like #
#############################################
"""
{
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "Let's use a tool.",
        "tool_calls": [
          {
            "id": "call_abc123",
            "type": "function",
            "function": {
              "name": "get_weather",
              "arguments": "{\"location\": \"Delhi\"}"
            }
          }
        ]
      }
    }
  ]
}

"""

## How Tool calling WOrks

In [ ]:
# ✅ STEP 1: Define Your Python Function
"""   
def get_weather(location: str) -> str:
    #Returns the current weather of the given location.  
    return f"The weather in {location} is sunny 🌞"                    
"""

#✅ STEP 2: Wrap It as a Tool (OpenAI format)
"""
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Returns the current weather of the given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                },
                "required": ["location"]
            }
        }
    }
]

"""

# ✅ STEP 3: Make API Call with Tools
""" 
response = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {"role": "user", "content": "What's the weather in Delhi?"}
    ],
    tools=tools
)

"""


# ✅ STEP 4: Handle tool_calls from the Response
"""
tool_calls = response.choices[0].message.tool_calls
"""
##################################################
# If tool calls exist, process them:
##################################################
"""
for tool_call in tool_calls:
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    # Call actual Python function
    if function_name == "get_weather":
        result = get_weather(**arguments)  # get_weather(location="Delhi")

"""

# 🧠 BONUS: Sending tool response back to LLM (optional)
"""
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": result  # The result of get_weather()
})

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)
"""







'\nmessages.append({\n    "role": "tool",\n    "tool_call_id": tool_call.id,\n    "content": result  # The result of get_weather()\n})\n\nresponse = client.chat.completions.create(\n    model="gpt-4o",\n    messages=messages\n)\n'

In [ ]:
"""
| Step                  | What You Do                                   |
| --------------------- | --------------------------------------------- |
| 1. Define Tool        | Write your Python function                    |
| 2. Wrap Tool          | Convert to JSON tool format                   |
| 3. Send API Call      | Pass `tools=[...]` to `.create()`             |
| 4. Check `tool_calls` | Read `response.choices[0].message.tool_calls` |
| 5. Run Tool           | Call the real Python function                 |
| 6. Respond (Optional) | Send result back via `role="tool"`            |

"""